In [1]:
# DEPENDENCIES ------>
import os
import pickle
import mediapipe as mp
import cv2

In [2]:

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)
data_dir = "./data_mydata"
data = []
labels = []


In [4]:
PROCESSED_DIR = "./processed"
if not os.path.exists(PROCESSED_DIR):
    os.makedirs(PROCESSED_DIR)


In [4]:
for dir_ in os.listdir(data_dir):
    for img_path in os.listdir(os.path.join(data_dir, dir_)):
        data_aux = []
        img = cv2.imread(os.path.join(data_dir, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x)
                    data_aux.append(y)
            data.append(data_aux)
            labels.append(dir_)
file = open("data.pickle", "wb")
pickle.dump({"data": data, "labels": labels}, file)
file.close()


In [3]:
# DEPENDENCIES ------>
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from keras.preprocessing.sequence import pad_sequences
import numpy as np


In [ ]:
data_dict = pickle.load(open("./data.pickle", "rb"))
data = np.asarray(data_dict["data"])
labels = np.asarray(data_dict["labels"])
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
score = accuracy_score(y_predict, y_test)

print("{}% classified correctly".format(score * 100))
f = open("model.p", "wb")
pickle.dump({"model": model}, f)
f.close()

# # Check the lengths of the sequences in data
# sequence_lengths = [len(seq) for seq in data_dict["data"]]

# # Print the unique lengths
# print("Unique sequence lengths: ", set(sequence_lengths))